In [1]:
# this script is intended to help generate install_dependecy scripts for different platforms


In [2]:
import os
import glob
import json
import datetime

In [3]:
def bash_header(install_prefix='/opt', workdir='/tmp'):
    current_year = datetime.datetime.now().year
    current_date = str(datetime.datetime.now())
    return '''#!/usr/bin/env bash
# (c) Tuplex team 2017-{}
# auto-generated on {}
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to {} by default'''.format(current_year, current_date, install_prefix) + \
'\n' + '''
# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX=${{PREFIX:-{}}}
WORKDIR=${{WORKDIR:-{}}}

echo ">> Installing packages into ${{PREFIX}}"
mkdir -p $PREFIX && chmod 0755 $PREFIX
mkdir -p $PREFIX/sbin
mkdir -p $PREFIX/bin
mkdir -p $PREFIX/share
mkdir -p $PREFIX/include
mkdir -p $PREFIX/lib

echo ">> Files will be downloaded to ${{WORKDIR}}/tuplex-downloads"
WORKDIR=$WORKDIR/tuplex-downloads
mkdir -p $WORKDIR

PYTHON_EXECUTABLE=${{PYTHON_EXECUTABLE:-python3}}
PYTHON_BASENAME="$(basename -- $PYTHON_EXECUTABLE)"
PYTHON_VERSION=$(${{PYTHON_EXECUTABLE}} --version)
echo ">> Building dependencies for ${{PYTHON_VERSION}}"

'''.format(install_prefix, workdir)

In [4]:
with open('test.sh', 'w') as fp:
    fp.write(bash_header())

In [5]:
print(bash_header())

#!/usr/bin/env bash
# (c) Tuplex team 2017-2022
# auto-generated on 2022-09-07 00:49:58.162530
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to /opt by default

# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX=${PREFIX:-/opt}
WORKDIR=${WORKDIR:-/tmp}

echo ">> Installing packages into ${PREFIX}"
mkdir -p $PREFIX && chmod 0755 $PREFIX
mkdir -p $PREFIX/sbin
mkdir -p $PREFIX/bin
mkdir -p $PREFIX/share
mkdir -p $PREFIX/include
mkdir -p $PREFIX/lib

echo ">> Files will be downloaded to ${WORKDIR}/tuplex-downloads"
WORKDIR=$WORKDIR/tuplex-downloads
mkdir -p $WORKDIR

PYTHON_EXECUTABLE=${PYTHON_EXECUTABLE:-python3}
PYTHON_BASENAME="$(basename -- $PYTHON_EXECUTABLE)"
PYTHON_VERSION=$(${PYTHON_EXECUTABLE} --version)
echo ">> Building dependencies for ${PYTHON_VERSION}"




In [6]:
# configure here which versions to use for each platform
VERSIONS = {}
    
def configure_versions(osname):
    global VERSIONS
    
    VERSIONS['OS'] = osname
    
    VERSIONS['CMAKE_VERSION'] = '3.23.3'
    VERSIONS['YAMLCPP_VERSION']='0.6.3'
    VERSIONS['CELERO_VERSION']='2.8.3'
    VERSIONS['ANTLR_VERSION']='4.8'
    VERSIONS['AWSSDK_VERSION']='1.9.320'#'1.9.133'
    VERSIONS['AWSLAMBDACPP_VERSION']='0.2.6'
    VERSIONS['PCRE2_VERSION']='10.39'
    VERSIONS['PROTOBUF_VERSION']='3.21.5'
    VERSIONS['CLOUDPICKLE_VERSION']='<2.0.0'
    VERSIONS['BOOST_VERSION']='1.79.0'
    VERSIONS['GCC_VERSION']='10'
    VERSIONS['LLVM_VERSION']='9.0.1'

    VERSIONS['MONGODB_VERSION']='5.0'

    VERSIONS['WORKDIR']='/tmp'
    VERSIONS['PREFIX']='/opt'

    # for ubuntu 18.04 use gcc7
    if '18.04' in osname:
        VERSIONS['GCC_VERSION']='7'
    
    # for 20.04 use gcc10
    if '20.04' in osname:
        VERSIONS['GCC_VERSION']='10'
    # for ubuntu 22.04 use gcc11
    if '22.04' in osname:
        VERSIONS['GCC_VERSION']='11.2'

    # derived vars
    VERSIONS['GCC_VERSION_MAJOR']=VERSIONS['GCC_VERSION'].split('.')[0]
    CMAKE_VERSION=VERSIONS['CMAKE_VERSION']
    VERSIONS['CMAKE_VERSION_MAJOR'],VERSIONS['CMAKE_VERSION_MINOR'],VERSIONS['CMAKE_VERSION_PATCH']=tuple(CMAKE_VERSION.split('.'))

In [7]:
configure_versions('ubuntu:18.04')

In [8]:
def install_mongodb(osname='ubuntu:22.04'):
    MONGODB_VERSION=VERSIONS['MONGODB_VERSION']
    if osname == 'ubuntu:18.04':
        return  """apt-get update && apt-get install -y curl gnupg \\
&& curl -fsSL https://www.mongodb.org/static/pgp/server-{version}.asc | apt-key add - \\
&& echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/{version} multiverse" | tee /etc/apt/sources.list.d/mongodb-org-{version}.list \\
&& apt update \\
&& apt install -y mongodb-org""".format(version=MONGODB_VERSION)
    else:
        raise Exception('unknown os {}'.format(osname))
   

In [9]:
def tiny_bash_header():
    current_year = datetime.datetime.now().year
    current_date = str(datetime.datetime.now())
    return '#!/usr/bin/env bash\n#(c) 2017-{} Tuplex team\n\n'.format(current_year)

def write_bash_file(path, content, header=tiny_bash_header()):
    with open(path, 'w') as fp:
        fp.write(header)
        fp.write(content)
    os.chmod(path, 0o700) # rwx------

In [10]:
help(os.chmod)

Help on built-in function chmod in module posix:

chmod(path, mode, *, dir_fd=None, follow_symlinks=True)
    Change the access permissions of a file.
    
      path
        Path to be modified.  May always be specified as a str, bytes, or a path-like object.
        On some platforms, path may also be specified as an open file descriptor.
        If this functionality is unavailable, using it raises an exception.
      mode
        Operating-system mode bitfield.
      dir_fd
        If not None, it should be a file descriptor open to a directory,
        and path should be relative; path will then be relative to that
        directory.
      follow_symlinks
        If False, and the last element of the path is a symbolic link,
        chmod will modify the symbolic link itself instead of the file
        the link points to.
    
    It is an error to use dir_fd or follow_symlinks when specifying path as
      an open file descriptor.
    dir_fd and follow_symlinks may not be impleme

In [11]:
# write ubuntu 18.04 files
osname = 'ubuntu:18.04'

root_folder = 'ubuntu1804'
os.makedirs(root_folder, exist_ok=True)

write_bash_file(os.path.join(root_folder, 'install_mongodb.sh'), install_mongodb(osname))

In [12]:
%%file ubuntu1804/Dockerfile

FROM ubuntu:18.04
    
# build using docker build -t tuplex/ubuntu:1804 .

MAINTAINER Leonhard Spiegelberg "leonhard@brown.edu"

RUN mkdir -p /opt/sbin

ADD install_mongodb.sh /opt/sbin/install_mongodb.sh
RUN /opt/sbin/install_mongodb.sh

Overwriting ubuntu1804/Dockerfile


In [13]:
print(install_mongodb('ubuntu:18.04'))

apt-get update && apt-get install -y curl gnupg \
&& curl -fsSL https://www.mongodb.org/static/pgp/server-5.0.asc | apt-key add - \
&& echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/5.0 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-5.0.list \
&& apt update \
&& apt install -y mongodb-org


In [14]:
def apt_dependencies(osname='ubuntu:22.04'):
    
    GCC_VERSION_MAJOR=VERSIONS['GCC_VERSION_MAJOR']
    
    packages_dict = {'ubuntu:22.04': '''apt-utils dh-autoreconf libmagic-dev curl libxml2-dev vim build-essential libssl-dev zlib1g-dev libncurses5-dev \\
    libncursesw5-dev libreadline-dev libsqlite3-dev libgdbm-dev libdb5.3-dev \\
    libbz2-dev libexpat1-dev liblzma-dev tk-dev libffi-dev wget git libcurl4-openssl-dev python3-dev python3-pip openjdk-8-jre-headless''',
                     'ubuntu:20.04': '''software-properties-common dh-autoreconf curl build-essential wget git libedit-dev libz-dev \\
                   python3-yaml python3-pip pkg-config libssl-dev libcurl4-openssl-dev curl \\
                   uuid-dev libffi-dev libmagic-dev \\
                   doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \\
                   libgflags-dev libncurses-dev \\
                   openjdk-8-jdk libyaml-dev ninja-build gcc-{} g++-{} autoconf libtool m4
                     '''.format(GCC_VERSION_MAJOR, GCC_VERSION_MAJOR),
                    'ubuntu:18.04': '''build-essential wget git dh-autoreconf libxml2-dev \\
 autoconf curl automake libtool software-properties-common wget libedit-dev libz-dev \\
  python3-yaml pkg-config libssl-dev libcurl4-openssl-dev curl \\
  uuid-dev git python3.7 python3.7-dev python3-pip libffi-dev \\
  doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \\
  gcc-{} g++-{} libgflags-dev libncurses-dev \\
  awscli openjdk-8-jdk libyaml-dev libmagic-dev ninja-build
                    '''.format(GCC_VERSION_MAJOR, GCC_VERSION_MAJOR)}
    
    return 'apt update -y\n' + \
'''
apt-get install -y {}
  
ldconfig
export CC=gcc-{}
export CXX=g++-{}
'''.format(packages_dict[osname], GCC_VERSION_MAJOR, GCC_VERSION_MAJOR)

In [15]:
print(apt_dependencies('ubuntu:18.04'))

apt update -y

apt-get install -y build-essential wget git dh-autoreconf libxml2-dev \
 autoconf curl automake libtool software-properties-common wget libedit-dev libz-dev \
  python3-yaml pkg-config libssl-dev libcurl4-openssl-dev curl \
  uuid-dev git python3.7 python3.7-dev python3-pip libffi-dev \
  doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \
  gcc-7 g++-7 libgflags-dev libncurses-dev \
  awscli openjdk-8-jdk libyaml-dev libmagic-dev ninja-build
                    
  
ldconfig
export CC=gcc-7
export CXX=g++-7



In [16]:
def install_cmake():
    
    CMAKE_VERSION_MAJOR = VERSIONS['CMAKE_VERSION_MAJOR']
    CMAKE_VERSION_MINOR = VERSIONS['CMAKE_VERSION_MINOR']
    CMAKE_VERSION_PATCH = VERSIONS['CMAKE_VERSION_PATCH']
    
    return '''
# fetch recent cmake & install
CMAKE_VER_MAJOR={}
CMAKE_VER_MINOR={}
CMAKE_VER_PATCH={}'''.format(CMAKE_VERSION_MAJOR,CMAKE_VERSION_MINOR,CMAKE_VERSION_PATCH) + '''
CMAKE_VER="${CMAKE_VER_MAJOR}.${CMAKE_VER_MINOR}"
CMAKE_VERSION="${CMAKE_VER}.${CMAKE_VER_PATCH}"
URL=https://github.com/Kitware/CMake/releases/download/v${CMAKE_VERSION}/cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz
mkdir -p ${WORKDIR}/cmake && cd ${WORKDIR}/cmake &&
  curl -sSL $URL -o cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  tar -v -zxf cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  rm -f cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  cd cmake-${CMAKE_VERSION}-linux-x86_64 &&
  cp -rp bin/* ${PREFIX}/bin/ &&
  cp -rp share/* ${PREFIX}/share/ &&
  cd / && rm -rf ${WORKDIR}/cmake
    '''

In [17]:
print(install_cmake())


# fetch recent cmake & install
CMAKE_VER_MAJOR=3
CMAKE_VER_MINOR=23
CMAKE_VER_PATCH=3
CMAKE_VER="${CMAKE_VER_MAJOR}.${CMAKE_VER_MINOR}"
CMAKE_VERSION="${CMAKE_VER}.${CMAKE_VER_PATCH}"
URL=https://github.com/Kitware/CMake/releases/download/v${CMAKE_VERSION}/cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz
mkdir -p ${WORKDIR}/cmake && cd ${WORKDIR}/cmake &&
  curl -sSL $URL -o cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  tar -v -zxf cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  rm -f cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  cd cmake-${CMAKE_VERSION}-linux-x86_64 &&
  cp -rp bin/* ${PREFIX}/bin/ &&
  cp -rp share/* ${PREFIX}/share/ &&
  cd / && rm -rf ${WORKDIR}/cmake
    


In [18]:
def install_boost():
    # may need to include the following to work for boost python
    # # fix up for boost python a link
    # cd /usr/local/include/ && ln -s python3.6m python3.6 && cd - || exit
    
    BOOST_VERSION=VERSIONS['BOOST_VERSION']
    
    boost_reformatted = BOOST_VERSION.replace('.', '_')
    
    # boost has a good amount of issues, following will result in a bug for GCC
    # -> this line in $PREFIX/include/boost/thread/pthread/thread_data.hpp
    # #if PTHREAD_STACK_MIN > 0
    # should be replaced with
    # #ifdef PTHREAD_STACK_MIN
    
    boost_fix = " && sed -i 's/#if PTHREAD_STACK_MIN > 0/#ifdef PTHREAD_STACK_MIN/g' /opt/include/boost/thread/pthread/thread_data.hpp"
    return '''    
mkdir -p ${WORKDIR/boost}
    
# build incl. boost python
pushd ${WORKDIR/boost} && ''' + \
'wget https://boostorg.jfrog.io/artifactory/main/release/{}/source/boost_{}.tar.gz && tar xf boost_{}.tar.gz'.format(BOOST_VERSION, boost_reformatted, boost_reformatted, boost_reformatted) + \
' && cd ${{WORKDIR/boost}}/boost_{}'.format(boost_reformatted) + \
''' \\
&& ./bootstrap.sh --with-python=${PYTHON_EXECUTABLE} --prefix=${PREFIX} --with-libraries="thread,iostreams,regex,system,filesystem,python,stacktrace,atomic,chrono,date_time" \\
 && ./b2 cxxflags="-fPIC" link=static -j "$(nproc)" \\
 && ./b2 cxxflags="-fPIC" link=static install''' + boost_fix

In [19]:
print(install_boost())

    
mkdir -p ${WORKDIR/boost}
    
# build incl. boost python
pushd ${WORKDIR/boost} && wget https://boostorg.jfrog.io/artifactory/main/release/1.79.0/source/boost_1_79_0.tar.gz && tar xf boost_1_79_0.tar.gz && cd ${WORKDIR/boost}/boost_1_79_0 \
&& ./bootstrap.sh --with-python=${PYTHON_EXECUTABLE} --prefix=${PREFIX} --with-libraries="thread,iostreams,regex,system,filesystem,python,stacktrace,atomic,chrono,date_time" \
 && ./b2 cxxflags="-fPIC" link=static -j "$(nproc)" \
 && ./b2 cxxflags="-fPIC" link=static install && sed -i 's/#if PTHREAD_STACK_MIN > 0/#ifdef PTHREAD_STACK_MIN/g' /opt/include/boost/thread/pthread/thread_data.hpp


In [20]:
def install_protobuf():
    PROTOBUF_VERSION=VERSIONS['PROTOBUF_VERSION']
    
    code = 'mkdir -p ${WORKDIR}/protobuf && cd ${WORKDIR}/protobuf \\'
    
    code += '''\n&& curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v{minversion}/protobuf-cpp-{version}.tar.gz \\
&& tar xf protobuf-cpp-{version}.tar.gz \\
&& cd protobuf-{version} \\
&& ./autogen.sh && ./configure "CFLAGS=-fPIC" "CXXFLAGS=-fPIC" \\
&& make -j$(nproc) && make install && ldconfig'''.format(version=PROTOBUF_VERSION, minversion=PROTOBUF_VERSION.replace('3.', ''))
    
    return code



print(install_protobuf())

mkdir -p ${WORKDIR}/protobuf && cd ${WORKDIR}/protobuf \
&& curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v21.5/protobuf-cpp-3.21.5.tar.gz \
&& tar xf protobuf-cpp-3.21.5.tar.gz \
&& cd protobuf-3.21.5 \
&& ./autogen.sh && ./configure "CFLAGS=-fPIC" "CXXFLAGS=-fPIC" \
&& make -j$(nproc) && make install && ldconfig


In [21]:
def install_pcre2():
    PCRE2_VERSION=VERSIONS['PCRE2_VERSION']
    
    
    code = 'mkdir -p ${WORKDIR}/pcre2 && cd ${WORKDIR}/pcre2 \\'
    
    code += '''\n&& curl -LO https://github.com/PhilipHazel/pcre2/releases/download/pcre2-{version}/pcre2-{version}.zip \\
&& unzip pcre2-{version}.zip \\
&& rm pcre2-{version}.zip \\
&& cd pcre2-{version} \\
&& ./configure CFLAGS="-O2 -fPIC" --prefix=${{PREFIX}} --enable-jit=auto --disable-shared \\
&& make -j$(nproc) && make install'''.format(version=PCRE2_VERSION)
    
    return code


print(install_pcre2())

mkdir -p ${WORKDIR}/pcre2 && cd ${WORKDIR}/pcre2 \
&& curl -LO https://github.com/PhilipHazel/pcre2/releases/download/pcre2-10.39/pcre2-10.39.zip \
&& unzip pcre2-10.39.zip \
&& rm pcre2-10.39.zip \
&& cd pcre2-10.39 \
&& ./configure CFLAGS="-O2 -fPIC" --prefix=${PREFIX} --enable-jit=auto --disable-shared \
&& make -j$(nproc) && make install


In [22]:
def install_aws_sdk():
    # install BOTH aws sdk and LAMBDA runtime
    
    AWSSDK_VERSION=VERSIONS['AWSSDK_VERSION']
    AWSLAMBDACPP_VERSION=VERSIONS['AWSLAMBDACPP_VERSION']
    
    code = 'mkdir -p ${WORKDIR}/aws && cd ${WORKDIR}/aws \\'
    
    code += '''\n&&  git clone --recurse-submodules https://github.com/aws/aws-sdk-cpp.git \\
&& cd aws-sdk-cpp && git checkout tags/{version} && mkdir build && cd build \\
&& cmake -DCMAKE_BUILD_TYPE=Release -DUSE_OPENSSL=ON -DENABLE_TESTING=OFF -DENABLE_UNITY_BUILD=ON -DCPP_STANDARD=14 -DBUILD_SHARED_LIBS=OFF -DBUILD_ONLY="s3;core;lambda;transfer" -DCMAKE_INSTALL_PREFIX=${{PREFIX}} .. \\
&& make -j$(nproc) \\
&& make install'''.format(version=AWSSDK_VERSION)

    code += '\n\n#installing AWS Lambda C++ runtime\n'
    code += '\ncd ${WORKDIR}/aws \\\n'
    code += '''&& git clone https://github.com/awslabs/aws-lambda-cpp.git \\
&& cd aws-lambda-cpp \\
&& git fetch && git fetch --tags \\
&& git checkout v{version} \\
&& mkdir build \\
&& cd build \\
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${{PREFIX}} .. \\
&& make -j$(nproc) && make install'''.format(version=AWSLAMBDACPP_VERSION)
    
    return code


print(install_aws_sdk())

mkdir -p ${WORKDIR}/aws && cd ${WORKDIR}/aws \
&&  git clone --recurse-submodules https://github.com/aws/aws-sdk-cpp.git \
&& cd aws-sdk-cpp && git checkout tags/1.9.320 && mkdir build && cd build \
&& cmake -DCMAKE_BUILD_TYPE=Release -DUSE_OPENSSL=ON -DENABLE_TESTING=OFF -DENABLE_UNITY_BUILD=ON -DCPP_STANDARD=14 -DBUILD_SHARED_LIBS=OFF -DBUILD_ONLY="s3;core;lambda;transfer" -DCMAKE_INSTALL_PREFIX=${PREFIX} .. \
&& make -j$(nproc) \
&& make install

#installing AWS Lambda C++ runtime

cd ${WORKDIR}/aws \
&& git clone https://github.com/awslabs/aws-lambda-cpp.git \
&& cd aws-lambda-cpp \
&& git fetch && git fetch --tags \
&& git checkout v0.2.6 \
&& mkdir build \
&& cd build \
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${PREFIX} .. \
&& make -j$(nproc) && make install


In [23]:
def install_antlr():
    # install both ANTLR tool and C++ runtime
    ANTLR_VERSION=VERSIONS['ANTLR_VERSION']
    
    ANTLR_MAJMIN = '.'.join(ANTLR_VERSION.split('.')[:2])
    
    code = 'mkdir -p ${WORKDIR}/antlr && cd ${WORKDIR}/antlr \\\n'
    
    code += '''&& curl -O https://www.antlr.org/download/antlr-{majmin}-complete.jar \\
&& cp antlr-{majmin}-complete.jar ${{PREFIX}}/lib/ \\\n'''.format(majmin=ANTLR_MAJMIN)
    
    # install runtime
    code += '''&& curl -O https://www.antlr.org/download/antlr4-cpp-runtime-{majmin}-source.zip \\
&& unzip antlr4-cpp-runtime-{majmin}-source.zip -d antlr4-cpp-runtime \\
&& rm antlr4-cpp-runtime-{majmin}-source.zip \\
&& cd antlr4-cpp-runtime \\
&& mkdir build && cd build && cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${{PREFIX}} .. \\
&& make -j$(nproc) && make install'''.format(majmin=ANTLR_MAJMIN)
    
    return code


print(install_antlr())

mkdir -p ${WORKDIR}/antlr && cd ${WORKDIR}/antlr \
&& curl -O https://www.antlr.org/download/antlr-4.8-complete.jar \
&& cp antlr-4.8-complete.jar ${PREFIX}/lib/ \
&& curl -O https://www.antlr.org/download/antlr4-cpp-runtime-4.8-source.zip \
&& unzip antlr4-cpp-runtime-4.8-source.zip -d antlr4-cpp-runtime \
&& rm antlr4-cpp-runtime-4.8-source.zip \
&& cd antlr4-cpp-runtime \
&& mkdir build && cd build && cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${PREFIX} .. \
&& make -j$(nproc) && make install


In [24]:
def install_yaml():
    YAMLCPP_VERSION=VERSIONS['YAMLCPP_VERSION']
    
    # install yaml cpp
    code = 'mkdir -p ${WORKDIR}/yamlcpp && cd ${WORKDIR}/yamlcpp \\'
    
    code += '''\n&& git clone https://github.com/jbeder/yaml-cpp.git yaml-cpp \\
&& cd yaml-cpp \\
&& git checkout tags/yaml-cpp-{version} \\
&& mkdir build && cd build \\
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${{prefix}} -DYAML_CPP_BUILD_TESTS=OFF -DBUILD_SHARED_LIBS=OFF -DCMAKE_CXX_FLAGS="-fPIC" .. \\
&& make -j$(nproc) && make install'''.format(version=YAMLCPP_VERSION)
    
    return code

print(install_yaml())

mkdir -p ${WORKDIR}/yamlcpp && cd ${WORKDIR}/yamlcpp \
&& git clone https://github.com/jbeder/yaml-cpp.git yaml-cpp \
&& cd yaml-cpp \
&& git checkout tags/yaml-cpp-0.6.3 \
&& mkdir build && cd build \
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${prefix} -DYAML_CPP_BUILD_TESTS=OFF -DBUILD_SHARED_LIBS=OFF -DCMAKE_CXX_FLAGS="-fPIC" .. \
&& make -j$(nproc) && make install


In [25]:
def install_celero():
    CELERO_VERSION=VERSIONS['CELERO_VERSION']
    
    # install ycelero
    code = 'mkdir -p ${WORKDIR}/celero && cd ${WORKDIR}/celero \\'
    
    code += '''\n&&  git clone https://github.com/DigitalInBlue/Celero.git celero && cd celero \\
&& git checkout tags/v{version} \\
&& mkdir build && cd build \\
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${{PREFIX}} -DBUILD_SHARED_LIBS=OFF -DCMAKE_CXX_FLAGS="-fPIC -std=c++11" .. \\
&& make -j$(nproc) && make install'''.format(version=CELERO_VERSION)
    
    return code

print(install_celero())

mkdir -p ${WORKDIR}/celero && cd ${WORKDIR}/celero \
&&  git clone https://github.com/DigitalInBlue/Celero.git celero && cd celero \
&& git checkout tags/v2.8.3 \
&& mkdir build && cd build \
&& cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=${PREFIX} -DBUILD_SHARED_LIBS=OFF -DCMAKE_CXX_FLAGS="-fPIC -std=c++11" .. \
&& make -j$(nproc) && make install


In [26]:
def bashrc_helper():
    ANTLR_VERSION=VERSIONS['ANTLR_VERSION']
    PREFIX=VERSIONS['PREFIX']
    
    code = 'echo "Consider adding the following lines to your ~/.bash_profile, ~/.bashrc or other shell config file:"\n\n'
    # could print out the following hints for bashrc
    
    # # setup bash aliases
    # echo "alias antlr='java -jar /opt/lib/antlr-4.8-complete.jar'" >>"$HOME/.bashrc"
    # echo "alias grun='java org.antlr.v4.gui.TestRig'" >>"$HOME/.bashrc"

    # # update include/link paths to /opt
    # echo "export CPLUS_INCLUDE_PATH=/opt/include:\${CPLUS_INCLUDE_PATH}" >> "$HOME/.bashrc"
    # echo "export C_INCLUDE_PATH=/opt/include:\${C_INCLUDE_PATH}" >> "$HOME/.bashrc"
    # echo "export LD_LIBRARY_PATH=/opt/lib:\${LD_LIBRARY_PATH}" >> "$HOME/.bashrc"
    
    ANTLR_MAJMIN = '.'.join(ANTLR_VERSION.split('.')[:2])
    ANTLR_MAJ=ANTLR_VERSION.split('.')[0]
    
    code += """
alias antlr='java -jar /opt/lib/antlr-{}-complete.jar'
alias grun='java org.antlr.v{}.gui.TestRig'

export PATH={prefix}/bin:$PATH
export CPLUS_INCLUDE_PATH={prefix}/include:${{CPLUS_INCLUDE_PATH}}
export C_INCLUDE_PATH={prefix}/include:${{C_INCLUDE_PATH}}
export LD_LIBRARY_PATH={prefix}/lib:${{LD_LIBRARY_PATH}}
    """.format(ANTLR_MAJMIN, ANTLR_MAJ, prefix=PREFIX)
    return code

print(bashrc_helper())

echo "Consider adding the following lines to your ~/.bash_profile, ~/.bashrc or other shell config file:"


alias antlr='java -jar /opt/lib/antlr-4.8-complete.jar'
alias grun='java org.antlr.v4.gui.TestRig'

export PATH=/opt/bin:$PATH
export CPLUS_INCLUDE_PATH=/opt/include:${CPLUS_INCLUDE_PATH}
export C_INCLUDE_PATH=/opt/include:${C_INCLUDE_PATH}
export LD_LIBRARY_PATH=/opt/lib:${LD_LIBRARY_PATH}
    


In [27]:
def install_llvm():
    
    GCC_VERSION_MAJOR=VERSIONS['GCC_VERSION_MAJOR']
    LLVM_VERSION=VERSIONS['LLVM_VERSION']
    
    cxx_flags = '-std=c++11'
    
    # GCC11+ requires explicit include of limits etc. cf.https://stackoverflow.com/questions/4798936/numeric-limits-was-not-declared-in-this-scope-no-matching-function-for-call-t
    if int(GCC_VERSION_MAJOR) >= 11:
        cxx_flags += ' -include /usr/include/c++/11/limits'
    
    
    LLVM_VERSION_MAJOR=LLVM_VERSION.split('.')[0]
    workdir='${WORKDIR}/llvm'
    code = 'mkdir -p {} && cd {} && '.format(workdir, workdir)  + \
'''wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{version}/llvm-{version}.src.tar.xz \\
&& wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{version}/clang-{version}.src.tar.xz \\
&& tar xf llvm-{version}.src.tar.xz && tar xf clang-{version}.src.tar.xz \\'''.format(version=LLVM_VERSION)

    cmake_base_flags = '''-DLLVM_ENABLE_RTTI=ON -DLLVM_ENABLE_EH=ON \\
        -DLLVM_ENABLE_PROJECTS="clang" \\
         -DLLVM_TARGETS_TO_BUILD="X86" -DCMAKE_BUILD_TYPE=Release'''
    
    if int(LLVM_VERSION_MAJOR) < 10:
    
        # LLVM version < 10
        code += '''\n&& mkdir llvm{maj} && mv clang-{version}.src llvm{maj}/clang \\
    && mv llvm-{version}.src llvm{maj}/llvm-{version}.src \\
    && cd llvm{maj}'''.format(version=LLVM_VERSION, maj=LLVM_VERSION_MAJOR)
        
        cmake_flags = cmake_base_flags
        
    else:
        # LLVM version >= 10
        code += '''\n&& mkdir llvm{maj} && mv clang-{version}.src llvm{maj}/clang && mv cmake llvm{maj}/cmake \\
    && mv llvm-{version}.src llvm{maj}/llvm-{version}.src \\
    && cd llvm{maj}'''.format(version=LLVM_VERSION, maj=LLVM_VERSION_MAJOR)

        cmake_flags = cmake_base_flags + ' -DLLVM_INCLUDE_BENCHMARKS=OFF'
        
        
    code += ''' && mkdir build && cd build \\
&& cmake {cmake_flags} -DCMAKE_CXX_FLAGS="{cxx_flags}" \\
         -DCMAKE_INSTALL_PREFIX=/opt/llvm-{version} ../llvm-{version}.src \\
&& make -j "$(nproc)" && make install'''.format(cmake_flags=cmake_flags, version=LLVM_VERSION, cxx_flags=cxx_flags)
    
    return code

In [28]:
print(install_llvm())

mkdir -p ${WORKDIR}/llvm && cd ${WORKDIR}/llvm && wget https://github.com/llvm/llvm-project/releases/download/llvmorg-9.0.1/llvm-9.0.1.src.tar.xz \
&& wget https://github.com/llvm/llvm-project/releases/download/llvmorg-9.0.1/clang-9.0.1.src.tar.xz \
&& tar xf llvm-9.0.1.src.tar.xz && tar xf clang-9.0.1.src.tar.xz \
&& mkdir llvm9 && mv clang-9.0.1.src llvm9/clang \
    && mv llvm-9.0.1.src llvm9/llvm-9.0.1.src \
    && cd llvm9 && mkdir build && cd build \
&& cmake -DLLVM_ENABLE_RTTI=ON -DLLVM_ENABLE_EH=ON \
        -DLLVM_ENABLE_PROJECTS="clang" \
         -DLLVM_TARGETS_TO_BUILD="X86" -DCMAKE_BUILD_TYPE=Release -DCMAKE_CXX_FLAGS="-std=c++11" \
         -DCMAKE_INSTALL_PREFIX=/opt/llvm-9.0.1 ../llvm-9.0.1.src \
&& make -j "$(nproc)" && make install


In [29]:
def make_ubuntu_1804_req_file(path):
    
    configure_versions('ubuntu:18.04')
    
    WORKDIR=VERSIONS['WORKDIR']
    PREFIX=VERSIONS['PREFIX']
    
    # create Ubuntu 1804 req file
    with open(path, 'w') as fp:
        fp.write(bash_header().strip())
        
        fp.write('\necho ">> Installing all build dependencies for Tuplex under Ubuntu 18.04"\n')

        fp.write('\necho ">> Installing apt dependencies"\n')
        fp.write(apt_dependencies('ubuntu:18.04').strip())

        fp.write('\n\necho ">> Installing recent cmake"\n')
        fp.write(install_cmake().strip())

        # use for following commands, newly installed cmake!
        fp.write('\n\nexport PATH=$PREFIX/bin:$PATH\ncmake --version\n')

        fp.write('\n\necho ">> Installing Boost"\n')
        fp.write(install_boost().strip())

        fp.write('\n\necho ">> Installing LLVM"\n')
        fp.write(install_llvm().strip())

        fp.write('\n\necho ">> Installing PCRE2"\n')
        fp.write(install_pcre2().strip())

        fp.write('\n\necho ">> Installing Celero"\n')
        fp.write(install_celero().strip())

        fp.write('\n\necho ">> Installing YAMLCPP"\n')
        fp.write(install_yaml().strip())

        fp.write('\n\necho ">> Installing ANTLR"\n')
        fp.write(install_antlr().strip())

        fp.write('\n\necho ">> Installing protobuf"\n')
        fp.write(install_protobuf().strip())

        fp.write('\n\necho ">> Installing AWS SDK"\n')
        fp.write(install_aws_sdk().strip())

        fp.write('\n\necho ">> Cleaning/removing workdir {}"'.format(WORKDIR))
        fp.write('\nrm -rf ${WORKDIR}\n')
        fp.write('\necho "-- Done, all Tuplex requirements installed to {} --"\n'.format(PREFIX))
        
    os.chmod(path, 0o700)

In [30]:
make_ubuntu_1804_req_file('ubuntu1804/install_requirements.sh')

In [31]:
!open ubuntu1804

In [32]:
%%file ubuntu1804/Dockerfile

FROM ubuntu:18.04
    
# build using docker build -t tuplex/ubuntu:1804 .

MAINTAINER Leonhard Spiegelberg "leonhard@brown.edu"

RUN mkdir -p /opt/sbin

ADD install_mongodb.sh /opt/sbin/install_mongodb.sh
RUN /opt/sbin/install_mongodb.sh

ADD install_requirements.sh /opt/sbin/install_requirements.sh
RUN /opt/sbin/install_requirements.sh

Overwriting ubuntu1804/Dockerfile


In [33]:
# create Ubuntu 22.04 req file

with open('install_dependencies_ubuntu_22_04.sh', 'w') as fp:
    fp.write(bash_header().strip())
    
    fp.write('\necho ">> Installing apt dependencies"\n')
    fp.write(apt_dependencies().strip())
    
    fp.write('\n\necho ">> Installing recent cmake"\n')
    fp.write(install_cmake().strip())
    
    # use for following commands, newly installed cmake!
    fp.write('\n\nexport PATH=$PREFIX/bin:$PATH\ncmake --version\n')
    
    fp.write('\n\necho ">> Installing Boost"\n')
    fp.write(install_boost().strip())
    
    fp.write('\n\necho ">> Installing LLVM"\n')
    fp.write(install_llvm().strip())
    
    fp.write('\n\necho ">> Installing PCRE2"\n')
    fp.write(install_pcre2().strip())
    
    fp.write('\n\necho ">> Installing Celero"\n')
    fp.write(install_celero().strip())
    
    fp.write('\n\necho ">> Installing YAMLCPP"\n')
    fp.write(install_yaml().strip())
    
    fp.write('\n\necho ">> Installing ANTLR"\n')
    fp.write(install_antlr().strip())
    
    fp.write('\n\necho ">> Installing protobuf"\n')
    fp.write(install_protobuf().strip())
    
    fp.write('\n\necho ">> Installing AWS SDK"\n')
    fp.write(install_aws_sdk().strip())
    
    fp.write('\n\necho ">> Cleaning/removing workdir {}"'.format(WORKDIR))
    fp.write('\nrm -rf ${WORKDIR}\n')
    fp.write('\necho "-- Done, all Tuplex requirements installed to {} --"\n'.format(PREFIX))

NameError: name 'WORKDIR' is not defined

In [ ]:
# create Ubunti 22.04 req file

with open('install_llvm14_ubuntu_22_04.sh', 'w') as fp:
    fp.write(bash_header().strip())
    
    LLVM_VERSION='14.0.6'
    
    fp.write('\n\necho ">> Installing LLVM"\n')
    fp.write(install_llvm().strip())

    
    fp.write('\n\necho ">> Cleaning/removing workdir {}"'.format(WORKDIR))
    fp.write('\nrm -rf ${WORKDIR}\n')
    fp.write('\necho "-- Done, all Tuplex requirements installed to {} --"\n'.format(PREFIX))

In [ ]:
!cat install_dependencies_ubuntu_22_04.sh

In [ ]:
# create docker containers to build tuplex for ubuntu 18.04, 20.04, 22.04
# => this is to test the versions
# @TODO.

# TODO: need to update azure pipelines to use ubuntu 22.04 for CI.

In [ ]:
!pip3 install docker